In [1]:
dir

<function dir>

In [2]:
ls

 ドライブ D のボリューム ラベルは ボリューム です
 ボリューム シリアル番号は C64B-4FA7 です

 D:\izag8216.github.io\ML365\D47_365 のディレクトリ

2021/08/27  07:29    <DIR>          .
2021/08/27  07:29    <DIR>          ..
2021/08/27  07:29    <DIR>          .ipynb_checkpoints
2021/08/27  07:28                72 20210827_D47_ItomaruML365.ipynb
2021/08/19  07:32             1,082 sample.txt
2021/08/27  07:26               250 whatsapp.py
               3 個のファイル               1,404 バイト
               3 個のディレクトリ  482,136,567,808 バイトの空き領域


In [7]:
import re
import pandas as pd
import numpy as np
import emoji
from collections import Counter
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Extract Time
def date_time(s):
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result:
        return True
    return False

# Find Authors or Contacts
def find_author(s):
    s = s.split(":")
    if len(s)==2:
        return True
    else:
        return False

# Finding Messages
def getDatapoint(line):
    splitline = line.split(' - ')
    dateTime = splitline[0]
    date, time = dateTime.split(", ")
    message = " ".join(splitline[1:])
    if find_author(message):
        splitmessage = message.split(": ")
        author = splitmessage[0]
        message = " ".join(splitmessage[1:])
    else:
        author= None
    return date, time, author, message

In [8]:
data = []
conversation = 'sample.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        if not line:
            break
        line = line.strip()
        if date_time(line):
            if len(messageBuffer) > 0:
                data.append([date, time, author, ' '.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getDatapoint(line)
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [9]:
df = pd.DataFrame(data, columns=["Date", 'Time', 'Author', 'Message'])
df['Date'] = pd.to_datetime(df['Date'])

data = df.dropna()
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["Message"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["Message"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["Message"]]
print(data.head())

        Date     Time    Author             Message  Positive  Negative  \
0 2016-10-09  2:10 PM  Person 2                 Yes       1.0       0.0   
1 2016-10-09  3:10 PM  Person 1               When?       0.0       0.0   
2 2016-10-09  4:10 PM  Person 2  In an hour, maybe?       0.0       0.0   
3 2016-10-09  5:10 PM  Person 2            See you!       0.0       0.0   
4 2016-10-09  6:10 PM  Person 1           Person 2?       0.0       0.0   

   Neutral  
0      0.0  
1      1.0  
2      1.0  
3      1.0  
4      1.0  


In [10]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)

Neutral 🙂 
